<a href="https://colab.research.google.com/github/danjshaw/ece57000-finalProject/blob/main/BERT_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial Setup

In [1]:
!pip install datasets
!pip install evaluate
!pip freeze > requirements.txt
import torch
import torch.nn as nn
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00
cuda


# LoRA Implementation

Define a LoRA classes for replacing dense layers. LoRA layers have rank decomposition matrices based on a provided dense layer, rank, and alpha. LoRA modules containing the A and B matrices are separate that way the LoRA modules can be stored or interchanged and a common model can be used.

In [2]:
class LoraModule(nn.Module):
  def __init__(self, in_features, out_features, rank, alpha):
    super().__init__()
    self.scale = alpha / rank
    self.A = nn.Parameter(torch.randn(in_features, rank))
    self.B = nn.Parameter(torch.zeros(rank, out_features))

  def forward(self, x):
    return (self.scale * (x @ self.A @ self.B))

class LoraLinear(nn.Module):
  def __init__(self, linear, rank, alpha):
    super().__init__()
    if (isinstance(linear, LoraLinear)):
      self.linear = linear.linear
      self.lora = LoraModule(self.linear.in_features, self.linear.out_features, rank, alpha)

    else:
      self.linear = linear
      self.lora = LoraModule(self.linear.in_features, self.linear.out_features, rank, alpha)

  def forward(self, x):
    return self.linear(x) + self.lora(x)

Helper function for applying lora layers to a given model. Supports RoBERTa and DeBERTa.

Helper function for checking the total number of parameter and the total number of trainable parameters for a given model.

In [3]:
def get_trainable_parameters(model):
  trainable_parameters = 0
  parameters = 0
  for param in model.parameters():
    # count the total number of parameters
    parameters += param.numel()
    if param.requires_grad:
      # count the total number of trainable parameters
      trainable_parameters += param.numel()
  return parameters, trainable_parameters

# Fine tuning bert-tiny



Hyperparameters from [google-research/bert](https://github.com/google-research/bert):



> For each task, we selected the best fine-tuning hyperparameters from the lists below, and trained for 4 epochs:
> * batch sizes: 8, 16, 32, 64, 128
> * learning rates: 3e-4, 1e-4, 5e-5, 3e-5



In [4]:
epochs = 4
batch_sizes = [8, 16, 32, 64, 128]
learning_rates = [3e-4, 1e-4, 5e-5, 3e-5]

In [5]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, set_seed

set_seed(0)
raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [6]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [7]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
import evaluate

def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [12]:
from transformers import Trainer, TrainingArguments

results = []
for i, size in enumerate(batch_sizes):
  for j, rate in enumerate(learning_rates):
    result = {"batch_size": size, "learning_rate": rate}

    training_args = TrainingArguments(
        "test-trainer",
        eval_strategy="epoch",
        per_device_eval_batch_size=size,
        per_device_train_batch_size=size,
        num_train_epochs=epochs,
        learning_rate=rate,
        # disable_tqdm=True,
        report_to="none"
    )

    trainer = Trainer(
        model,
        training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        data_collator=data_collator,
        processing_class=tokenizer,
        compute_metrics=compute_metrics,
    )

    result['train_output'] = trainer.train()
    result['eval_output'] = trainer.evaluate()
    result['result'] = result['eval_output']['eval_accuracy']
    results.append(result)


batch_size=8, learning_rate=0.0003


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,6.262599,0.678922,0.777589
2,0.000000,6.682646,0.664216,0.754919
3,0.000000,4.278942,0.671569,0.757246
4,0.026300,3.763878,0.659314,0.748644


TrainOutput(global_step=1836, training_loss=0.008636967808592506, metrics={'train_runtime': 23.3087, 'train_samples_per_second': 629.464, 'train_steps_per_second': 78.769, 'total_flos': 2609679636960.0, 'train_loss': 0.008636967808592506, 'epoch': 4.0})


{'eval_loss': 3.763878107070923, 'eval_accuracy': 0.6593137254901961, 'eval_f1': 0.7486437613019892, 'eval_runtime': 0.7598, 'eval_samples_per_second': 536.982, 'eval_steps_per_second': 67.123, 'epoch': 4.0}
batch_size=8, learning_rate=0.0001


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,5.818925,0.669118,0.769231
2,0.000500,5.953907,0.676471,0.773973
3,0.002800,5.800941,0.647059,0.734317
4,0.000000,5.780823,0.654412,0.742230


...

TrainOutput(global_step=116, training_loss=1.4021956193855971e-08, metrics={'train_runtime': 6.9899, 'train_samples_per_second': 2099.029, 'train_steps_per_second': 16.595, 'total_flos': 3168505346640.0, 'train_loss': 1.4021956193855971e-08, 'epoch': 4.0})


{'eval_loss': 5.794428825378418, 'eval_accuracy': 0.6691176470588235, 'eval_f1': 0.7602131438721137, 'eval_runtime': 0.6354, 'eval_samples_per_second': 642.132, 'eval_steps_per_second': 6.295, 'epoch': 4.0}
batch_size=128, learning_rate=3e-05


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,5.780506,0.664216,0.758377
2,No log,5.765406,0.669118,0.763573
3,No log,5.764274,0.671569,0.765734
4,No log,5.772689,0.671569,0.765734


TrainOutput(global_step=116, training_loss=2.995443096630158e-09, metrics={'train_runtime': 6.9947, 'train_samples_per_second': 2097.588, 'train_steps_per_second': 16.584, 'total_flos': 3168505346640.0, 'train_loss': 2.995443096630158e-09, 'epoch': 4.0})


{'eval_loss': 5.772689342498779, 'eval_accuracy': 0.6715686274509803, 'eval_f1': 0.7657342657342657, 'eval_runtime': 0.608, 'eval_samples_per_second': 671.107, 'eval_steps_per_second': 6.579, 'epoch': 4.0}


In [77]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): LoraLinear(
                (linear): Linear(in_features=128, out_features=128, bias=True)
                (lora): LoraModule()
              )
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): LoraLinear(
                (linear): Linear(in_features=128, out_features=128, bias=True)
                (lora): LoraModule()
              )
              (dropout): Dropout(p=0.1, inplace=False)

In [76]:
rank = 1
alpha = 1
for i, layer in enumerate(model.bert.encoder.layer):
  s = layer.attention.self
  s.query = LoraLinear(s.query, rank, alpha)
  s.value = LoraLinear(s.value, rank, alpha)

for name, param in model.named_parameters():
  if 'A' in name or 'B' in name:
    param.requires_grad = True
  else:
    param.requires_grad = False

In [87]:
get_trainable_parameters(model)

(4386178, 4386178)